In [0]:
service_credential = dbutils.secrets.get(scope="db_sceret_scope",key="appid-secret")
directory_tenant_id = dbutils.secrets.get(scope="db_sceret_scope",key="tenant-directory-id")
app_id = dbutils.secrets.get(scope="db_sceret_scope",key="app-id")


configs = {"fs.azure.account.auth.type": "OAuth",
          "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
          "fs.azure.account.oauth2.client.id": app_id,
          "fs.azure.account.oauth2.client.secret": service_credential,
          "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/"+ directory_tenant_id +"/oauth2/token"
          }



In [0]:
dbutils.fs.mount(source = "abfss://de-tasks@detaskstorage.dfs.core.windows.net/",mount_point = "/mnt/unicorns",extra_configs = configs)

Out[7]: True

In [0]:
unicorns_df = spark.read.format("com.crealytics.spark.excel").option("header", True).option("inferSchema", True) \
    .load('/mnt/unicorns/world-unicorns/Global-Unicorns.xlsx')



In [0]:
import pyspark.sql.functions as f

In [0]:
display(unicorns_df.show(10))

+------------+--------------+-------------------+--------------+-------------+--------------------+--------------------+
|     Company|Valuation ($B)|        Date Joined|       Country|         City|            Industry|    Select Investors|
+------------+--------------+-------------------+--------------+-------------+--------------------+--------------------+
|   ByteDance|         140.0|2017-04-07 00:00:00|         China|      Beijing|Artificial intell...|Sequoia Capital C...|
|      SpaceX|         127.0|2012-12-01 00:00:00| United States|    Hawthorne|               Other|Founders Fund, Dr...|
|       SHEIN|         100.0|2018-07-03 00:00:00|         China|     Shenzhen|E-commerce & dire...|Tiger Global Mana...|
|      Stripe|          95.0|2014-01-23 00:00:00| United States|San Francisco|             Fintech|Khosla Ventures, ...|
|Checkout.com|          40.0|2019-05-02 00:00:00|United Kingdom|       London|             Fintech|Tiger Global Mana...|
|       Canva|          40.0|201

In [0]:
unicorns_df.printSchema()

root
 |-- Company: string (nullable = true)
 |-- Valuation ($B): double (nullable = true)
 |-- Date Joined: timestamp (nullable = true)
 |-- Country: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Industry: string (nullable = true)
 |-- Select Investors: string (nullable = true)



In [0]:
unicorns_df.count()

Out[8]: 1191

In [0]:
unicorns_df = unicorns_df.drop('Select Investors')

In [0]:
unicorns_df.show()

+-------------+--------------+-------------------+--------------+-------------+--------------------+
|      Company|Valuation ($B)|        Date Joined|       Country|         City|            Industry|
+-------------+--------------+-------------------+--------------+-------------+--------------------+
|    ByteDance|         140.0|2017-04-07 00:00:00|         China|      Beijing|Artificial intell...|
|       SpaceX|         127.0|2012-12-01 00:00:00| United States|    Hawthorne|               Other|
|        SHEIN|         100.0|2018-07-03 00:00:00|         China|     Shenzhen|E-commerce & dire...|
|       Stripe|          95.0|2014-01-23 00:00:00| United States|San Francisco|             Fintech|
| Checkout.com|          40.0|2019-05-02 00:00:00|United Kingdom|       London|             Fintech|
|        Canva|          40.0|2018-01-08 00:00:00|     Australia|  Surry Hills|Internet software...|
|    Instacart|          39.0|2014-12-30 00:00:00| United States|San Francisco|Supply chain

In [0]:
unicorns_df = unicorns_df.toDF(*(c.replace(' ', '_') for c in unicorns_df.columns))

In [0]:
for col in unicorns_df.columns:
    unicorns_df = unicorns_df.withColumnRenamed(col, col.lower())

In [0]:

unicorns_df.show(3)

+---------+--------------+-------------------+-------------+---------+--------------------+
|  company|valuation_($b)|        date_joined|      country|     city|            industry|
+---------+--------------+-------------------+-------------+---------+--------------------+
|ByteDance|         140.0|2017-04-07 00:00:00|        China|  Beijing|Artificial intell...|
|   SpaceX|         127.0|2012-12-01 00:00:00|United States|Hawthorne|               Other|
|    SHEIN|         100.0|2018-07-03 00:00:00|        China| Shenzhen|E-commerce & dire...|
+---------+--------------+-------------------+-------------+---------+--------------------+
only showing top 3 rows



In [0]:
unicorns_df = unicorns_df.withColumn('year', f.split('date_joined', '-').getItem(0))

In [0]:
unicorns_df.show(3)

+---------+--------------+-------------------+-------------+---------+--------------------+----+
|  company|valuation_($b)|        date_joined|      country|     city|            industry|year|
+---------+--------------+-------------------+-------------+---------+--------------------+----+
|ByteDance|         140.0|2017-04-07 00:00:00|        China|  Beijing|Artificial intell...|2017|
|   SpaceX|         127.0|2012-12-01 00:00:00|United States|Hawthorne|               Other|2012|
|    SHEIN|         100.0|2018-07-03 00:00:00|        China| Shenzhen|E-commerce & dire...|2018|
+---------+--------------+-------------------+-------------+---------+--------------------+----+
only showing top 3 rows



In [0]:
unicorns_df.coalesce(1).write.csv('dbfs:/mnt/unicorns/global_unicorns_clean', mode='overwrite')

In [0]:
dbutils.fs.unmount("/mnt/unicorns")

/mnt/unicorns has been unmounted.
Out[8]: True